In [1]:
import importlib

#Py-ART, simply the best sowftware around.. Give those guys a grant
import cartopy
import pyart
#timezone info
import pytz
import glob
import sys
#plotting
from matplotlib import pyplot as plt

from gc import collect
#plotting on a maop

import operator

import xarray as xr
import netCDF4

import scipy.ndimage as ndimage

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np
from cmac import cmac, quicklooks, get_sounding_times, get_sounding_file_name, config, cmac_processing
import copy
import matplotlib.colors as mc
import matplotlib.ticker as mt
import matplotlib.font_manager as fm
from datetime import datetime

from dask.distributed import Client, progress, metrics, LocalCluster


import os

%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/scollis/anaconda3/envs/daskenv/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/home/scollis/anaconda3/envs/daskenv/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
# 24 may 20z
soundings_directory = '/lustre/or-hydra/cades-arm/proj-shared/data_transfer/sgp/sgpsondewnpnC1.b1/'
radar_directory = '/lustre/or-hydra/cades-arm/proj-shared/scanning_collab/kdp_sec/ppi/2018/0524/'
inf = 'sgpxsaprppiI5.20180524.204213.iris'
inf2 = 'sgpxsaprppiI5.20180524.173213.iris'
#radar = pyart.io.read(radar_directory + inf2)

In [3]:
ex_dir = '/lustre/or-hydra/cades-arm/proj-shared/scanning_collab/'
files_I5_ppi = [f for f in glob.glob(ex_dir + "**/sgpxsaprppiI5*.iris", recursive=True)]
files_I6_ppi = [f for f in glob.glob(ex_dir + "**/sgpxsaprppiI6*.iris", recursive=True)]
files_I4_ppi = [f for f in glob.glob(ex_dir + "**/sgpxsaprppiI4*.iris", recursive=True)]

files_I4_ppi.sort()
files_I5_ppi.sort()
files_I6_ppi.sort()


In [12]:
#lets package
def do_cmac_i5_sec(filename):
    from cmac import cmac, quicklooks, get_sounding_times, get_sounding_file_name, config, cmac_processing
    def file_formatter(stime, site, scanmode, base, expr):
        #base/year/monthday
        
        mday = stime.strftime('%m%d')
        odir = os.path.join(base,
                            expr.lower(),
                            scanmode,
                            stime.strftime('%Y'),
                            mday)
        fname1 = 'sgpxsapr' + scanmode + site + stime.strftime('.%Y%m%d.%H%M%S')
        return odir, fname1
    
    verbose = True
    experiment = 'xsapr_i5_ppi'
    meta_config = config.get_metadata(experiment)
    cmac_config = config.get_cmac_values(experiment)
    field_config = config.get_field_names(experiment)


    temp_field = field_config['temperature']
    alt_field = field_config['altitude']
    vel_field = field_config['velocity']

    #field_config['reflectivity'] = 'uncorrected_reflectivity_h'
    #field_config['phidp'] = 'uncorrected_differential_phase'

    ref_offset = cmac_config['ref_offset']
    self_const = cmac_config['self_const']
    
    field_config['phidp'] = 'differential_phase'
    soundings_directory = '/lustre/or-hydra/cades-arm/proj-shared/data_transfer/sgp/sgpsondewnpnC1.b1/'
    try:
        radar = pyart.io.read(filename)
        index_at_start = radar.sweep_start_ray_index['data'][0]
        time_at_start_of_radar = pyart.io.cfradial.netCDF4.num2date(radar.time['data'][index_at_start], 
                                          radar.time['units'])

        
        odir, fnout = file_formatter(time_at_start_of_radar, 'I5', 'sec', 
                                     '/lustre/or-hydra/cades-arm/proj-shared/scanning_collab', 'processed')
        print(odir)
        print(fnout)
        filenameout = os.path.join(odir, fnout) + '.nc'
        
        exists = os.path.isfile(filenameout)
        if exists:
            del radar
            radar = pyart.io.read(filenameout)
            cat_dict = {}
            for pair_str in radar.fields['gate_id']['notes'].split(','):
                cat_dict.update(
                    {pair_str.split(':')[1]:int(pair_str.split(':')[0])})
            
            tgates = float(radar.ngates*radar.nrays)
            zdat = radar.fields['gate_id']['data']
            rain = float(len(np.where(zdat == cat_dict['rain'])[0]))/tgates
            snow = float(len(np.where(zdat == cat_dict['snow'])[0]))/tgates
            rv = (time_at_start_of_radar, rain, snow)
            del radar
        
        else:
            sonds = os.listdir(soundings_directory)
            sonds.sort()
            mydts = [datetime.strptime(thiss.split('.')[2]+thiss.split('.')[3], '%Y%m%d%H%M%S') for thiss in sonds]
            sfn = 'sgpsondewnpnC1.b1'
            # Retrieve closest sonde in time to the time of the radar file.
            closest_time = min(
                mydts, key=lambda d: abs(d - time_at_start_of_radar))
            sonde_file = get_sounding_file_name(
                soundings_directory, sfn, closest_time)
            ncf_sounding = netCDF4.Dataset(sonde_file+'.v0')
            cfld = copy.deepcopy(radar.fields['reflectivity'])
            cfld['data'] = cfld['data']* 0
            cfld['data'][:, 0:10] = 1

            cfld['standard_name'] = 'clutter'
            cfld['long_name'] = 'clutter field'
            cfld['units'] = 'unitless'
            radar.add_field('ground_clutter', cfld, replace_existing=True)

            del cfld

            if 'clutter_mask_z_for_texture' not in cmac_config.keys():
                    cmac_config['clutter_mask_z_for_texture'] = False

            if cmac_config['clutter_mask_z_for_texture']:
                masked_vr = copy.deepcopy(radar.fields[vel_field])
                masked_vr['data'] = np.ma.masked_where(radar.fields['ground_clutter']['data'] == 1, masked_vr['data'])
                masked_vr['data'][radar.fields['ground_clutter']['data'] == 1] = np.nan
                radar.add_field('clutter_masked_velocity', masked_vr, replace_existing=True)

                texture = cmac_processing.get_texture(radar, 'clutter_masked_velocity')
                texture['data'][np.isnan(texture['data'])] = 0.0
            else:
                texture = cmac_processing.get_texture(radar, vel_field)

            z_dict, temp_dict = pyart.retrieve.map_profile_to_gates(ncf_sounding.variables[temp_field][:], 
                                                                    ncf_sounding.variables[alt_field][:], 
                                                                    radar)


            snr = pyart.retrieve.calculate_snr_from_reflectivity(radar)

            radar.add_field('sounding_temperature', temp_dict, replace_existing=True)
            radar.add_field('height', z_dict, replace_existing=True)
            radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)
            radar.add_field('velocity_texture', texture, replace_existing=True)

            del texture, snr, z_dict, temp_dict


            rhv_field = field_config['cross_correlation_ratio']
            ncp_field = field_config['normalized_coherent_power']

            if 'mbfs' not in cmac_config:
                cmac_config['mbfs'] = None

            if 'hard_const' not in cmac_config:
                cmac_config['hard_const'] = None


            my_fuzz, _ = cmac_processing.do_my_fuzz(radar, rhv_field, ncp_field, tex_start=2.5,
                                    tex_end=5., 
                                    custom_mbfs=cmac_config['mbfs'],
                                    custom_hard_constraints=cmac_config['hard_const'],
                                    verbose=True)

            radar.add_field('gate_id', my_fuzz,
                            replace_existing=True)

            del my_fuzz

            if 'ground_clutter' in radar.fields.keys():
                # Adding fifth gate id, clutter.
                clutter_data = radar.fields['ground_clutter']['data']
                gate_data = radar.fields['gate_id']['data']
                radar.fields['gate_id']['data'][clutter_data == 1] = 5
                notes = radar.fields['gate_id']['notes']
                radar.fields['gate_id']['notes'] = notes + ',5:clutter'
                radar.fields['gate_id']['valid_max'] = 5
            cat_dict = {}
            for pair_str in radar.fields['gate_id']['notes'].split(','):
                cat_dict.update(
                    {pair_str.split(':')[1]:int(pair_str.split(':')[0])})

            if verbose:
                print('##    gate_id')

            cat_dict = {}
            for pair_str in radar.fields['gate_id']['notes'].split(','):
                cat_dict.update(
                    {pair_str.split(':')[1]:int(pair_str.split(':')[0])})


            #HARD constraint for lowz
            lowz_regs = np.where(radar.fields['reflectivity']['data'] < -10)
            radar.fields['gate_id']['data'][lowz_regs] = cat_dict['no_scatter']
            del lowz_regs

            cmac_gates = pyart.correct.GateFilter(radar)
            cmac_gates.exclude_all()
            cmac_gates.include_equal('gate_id', cat_dict['rain'])
            cmac_gates.include_equal('gate_id', cat_dict['melting'])
            cmac_gates.include_equal('gate_id', cat_dict['snow'])

            cmac_gates_hard = pyart.correct.GateFilter(radar)
            cmac_gates_hard.exclude_all()
            cmac_gates_hard.include_equal('gate_id', cat_dict['rain'])
            cmac_gates_hard.include_equal('gate_id', cat_dict['melting'])
            cmac_gates_hard.include_equal('gate_id', cat_dict['snow'])
            cmac_gates_hard.exclude_below('reflectivity', 0)

            dspcf = pyart.correct.despeckle_field(radar,'differential_phase', gatefilter=cmac_gates_hard )

            #radar.gate_altitude['data']
            #radar.gate_altitude['data'] = radar.gate_altitude['data'].data

            # Create a simulated velocity field from the sonde object.
            u_field = field_config['u_wind']
            v_field = field_config['v_wind']
            u_wind = ncf_sounding.variables[u_field][:]
            v_wind = ncf_sounding.variables[v_field][:]
            alt_field = field_config['altitude']
            sonde_alt = ncf_sounding.variables[alt_field][:]
            profile = pyart.core.HorizontalWindProfile.from_u_and_v(
                sonde_alt, u_wind, v_wind)
            sim_vel = pyart.util.simulated_vel_from_profile(radar, profile)
            radar.add_field('simulated_velocity', sim_vel, replace_existing=True)

            corr_vel = pyart.correct.dealias_region_based(
                radar, vel_field=vel_field, ref_vel_field='simulated_velocity',
                keep_original=False, gatefilter=cmac_gates, centered=True)

            radar.add_field('corrected_velocity', corr_vel, replace_existing=True)

            del corr_vel, sim_vel

            print('##    corrected_velocity')
            print('##    simulated_velocity')

            fzl = cmac_processing.get_melt(radar)

            unf = pyart.correct.phase_proc.get_phidp_unf_gf(radar, dspcf,
                                                            phidp_field=field_config['phidp'],
                                                           sys_phase=300., nowrap=100,
                                                           ncpts=20)

            radar.add_field_like('differential_phase', 'unfolded_differential_phase', 
                                 unf, replace_existing=True)

            # Calculating differential phase fields.
            phidp, kdp = pyart.correct.phase_proc_lp_gf(
                radar, gatefilter=dspcf, offset=ref_offset, debug=True,
                nowrap=150, fzl=10000, self_const=self_const, phidp_field=field_config['phidp'])
            phidp_filt, kdp_filt = cmac_processing.fix_phase_fields(
                copy.deepcopy(kdp), copy.deepcopy(phidp), radar.range['data'],
                cmac_gates)

            radar.add_field('corrected_differential_phase', phidp,
                            replace_existing=True)
            radar.add_field('filtered_corrected_differential_phase', phidp_filt,
                            replace_existing=True)
            radar.add_field('corrected_specific_diff_phase', kdp,
                            replace_existing=True)
            radar.add_field('filtered_corrected_specific_diff_phase', kdp_filt,
                            replace_existing=True)

            del kdp_filt, kdp, phidp_filt, phidp, unf

            print('##    corrected_specific_diff_phase')
            print('##    filtered_corrected_specific_diff_phase')
            print('##    corrected_differential_phase')
            print('##    filtered_corrected_differential_phase')

            vpkdp, vpphidp = pyart.retrieve.kdp_proc.kdp_vulpiani(radar, 
                                                              gatefilter=dspcf, 
                                                              fill_value=None, 
                                                              psidp_field=None, 
                                                              kdp_field=None, 
                                                              phidp_field='unfolded_differential_phase', 
                                                              band='X', 
                                                              windsize=10, 
                                                              n_iter=10, 
                                                              interp=False, 
                                                              prefilter_psidp=False, 
                                                              filter_opt=None, parallel=False)

            radar.add_field('vulp_differential_phase', vpphidp,
                        replace_existing=True)
            radar.add_field('vulp_specific_diff_phase', vpkdp,
                            replace_existing=True)

            del vpkdp
            del vpphidp

            print('##    Vulpani')
            mkdp, mkphf, mkphr = pyart.retrieve.kdp_proc.kdp_maesaka(radar, 
                                                                 gatefilter=dspcf, 
                                                                 method='cg', 
                                                                 backscatter=None,
                                                                 Clpf=1.0, 
                                                                 length_scale=None, 
                                                                 first_guess=0.01,
                                                                 finite_order='low',
                                                                 fill_value=None, 
                                                                 proc=1, 
                                                                 psidp_field=None, 
                                                                 kdp_field=None, 
                                                                 phidp_field='unfolded_differential_phase',
                                                                 debug=False, 
                                                                 verbose=False)


            radar.add_field('mes_differential_phase_forward', mkphf,
                            replace_existing=True)
            radar.add_field('mes_differential_phase_reverse', mkphr,
                            replace_existing=True)

            radar.add_field('mes_specific_diff_phase', mkdp,
                            replace_existing=True)


            del mkdp
            del mkphf
            del mkphr

            print('##    Measaka')
            try:
                os.makedirs(odir)
            except:
                pass
            pyart.io.write_cfradial(filenameout, radar)

            ncf_sounding.close()
            tgates = float(radar.ngates*radar.nrays)
            zdat = radar.fields['gate_id']['data']

            rain = float(len(np.where(zdat == cat_dict['rain'])[0]))/tgates
            snow = float(len(np.where(zdat == cat_dict['snow'])[0]))/tgates
            del radar, cmac_gates, cmac_gates_hard, dspcf
            rv = (time_at_start_of_radar, rain, snow)
    
    except Exception as e:
        rv = 'Error ' + str(e)
    
    collect()
    return rv
    
    
    

    

In [5]:
len(files_I5_ppi)

13307

In [6]:
#client.restart()
client = Client('arm-jupyter.ornl.gov:5555')


In [ ]:
client

In [8]:
my_data = []

In [9]:
future = client.map(do_cmac_i5_sec, files_I5_ppi)


In [10]:
progress(future)

VBox()

In [ ]:
this_data = client.gather(future)
my_data.append(this_data)


In [40]:
future[6000]

<Future: status: finished, type: tuple, key: do_cmac_i5_sec-4ca0fffd4abbf28c05ccc5b4c92bb589>

In [28]:
client.gather(future[0])

"Error local variable 'cat_dict' referenced before assignment"

Client Scheduler: tcp://arm-jupyter.ornl.gov:5555 Dashboard: http://arm-jupyter.ornl.gov:8787/status,Cluster Workers: 30 Cores: 180 Memory: 1.28 TB


In [49]:

#simple deletes
#1377
#1hr 1min

#comprehensive deletes
#1538
#1hr 1 min 

#best effort 
#3411
#1hr 1 min



In [ ]:
future = client.map(do_cmac_i5_sec, files_I5_ppi[0:2000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[2000:4000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[4000:6000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[6000:8000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[8000:10000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[10000:12000])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
future = client.map(do_cmac_i5_sec, files_I5_ppi[12000:-1])
this_data = client.gather(future)
my_data.append(this_data)
client.restart()
client = Client('arm-jupyter.ornl.gov:5555')
print('1')
